Introduction

Import statements

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
here = os.getcwd()

regions_one_through_six = F"{here}/data/2020/survey_locations_map_id.csv"
locations = pd.read_csv(regions_one_through_six)
locations = locations[~locations.place.isin(['buren-2', 'taubenloch-2', 'taubenloch-1'])]
loc_map = locations[['place', 'ws']].set_index('place')


afilename = here+"/output/mostrecent/hd_samples_2020.csv"
some_data = pd.read_csv(afilename)
d = some_data.copy()

# watch_lists
watch_lists = pd.read_csv(F"{here}/output/aggregated_list.csv")
watch_lists.set_index('species_slug', inplace=True)


# this location did not show up in the location key
d = d[~d.place_slug.isin(['buren-2', 'taubenloch-2', 'taubenloch-1'])]
d['ws'] = d.place_slug.map(lambda x: loc_map.loc[x][0])

In [3]:
watch_lists

,BL,WL,under_sampled,red_list,gbif,lists
species_slug,,,,,,
abies-alba,0,0,0,1,1,['red_list' 'gbif']
abies-nordmanniana,0,0,0,0,1,['gbif']
abutilon-theophrasti,1,0,0,0,1,['BL' 'gbif']
acacia-dealbata,0,1,0,0,1,['WL' 'gbif']
acalypha-australis,0,0,0,0,1,['gbif']
...,...,...,...,...,...,...
xeranthemum-inapertum,0,0,0,1,1,['red_list' 'gbif']
yucca-filamentosa,0,0,0,0,1,['gbif']
zannichellia-palustris,0,0,0,1,1,['red_list' 'gbif']


In [4]:
# make a loc date column
d['loc_date'] = list(zip(d.place_slug, d.date))
# datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
def check_dates(x):
    try:
        data = dt.datetime.strptime(x, "%Y-%m-%d")
    except:
        data = 'no date'
    return data
d['date_stamp'] = d['date'].map(lambda x: check_dates(x))
d['month'] = d.date_stamp.dt.month

first_date = d.date.min()
last_date = d.date.max()

print("the first observation was madte on "+ f"{first_date}"+ " and the final observation was made on " + f"{last_date}" + ".")


print(F"There are {len(d[d.date_stamp == 'no date'])} records with no valid date")
print(F"\nThese are the months of the year that sampling occurred:\n{d.month.unique()}\n")

the first observation was made on 2020-06-03 and the final observation was made on 2020-09-28.
There are 0 records with no valid date

These are the months of the year that sampling occurred:
[9 7 6 8]



In [5]:
som_151 = d[d.ws == 151].copy()

nsamps = len(som_151.loc_date.unique())
nspecs = len(som_151.genus_species.unique())

# persample

# genus_species per sample
dt_151 = som_151.groupby(['loc_date', 'place', 'ws'], as_index=False).genus_species.nunique()

In [6]:
watch_lists

,BL,WL,under_sampled,red_list,gbif,lists
species_slug,,,,,,
abies-alba,0,0,0,1,1,['red_list' 'gbif']
abies-nordmanniana,0,0,0,0,1,['gbif']
abutilon-theophrasti,1,0,0,0,1,['BL' 'gbif']
acacia-dealbata,0,1,0,0,1,['WL' 'gbif']
acalypha-australis,0,0,0,0,1,['gbif']
...,...,...,...,...,...,...
xeranthemum-inapertum,0,0,0,1,1,['red_list' 'gbif']
yucca-filamentosa,0,0,0,0,1,['gbif']
zannichellia-palustris,0,0,0,1,1,['red_list' 'gbif']


In [7]:
species_151 = som_151.species_slug.unique()
watch_lists_specs = watch_lists.index

[x for x in species_151 if x not in watch_lists_specs]

['rubus-rubus',
 'tripleurospermum-maritimum',
 'anemone-hupehensis',
 'packera-aurea',
 'ligustrum-sinense',
 'eupatorium-serotinum',
 'salix-nigra',
 'phlox-pilosa',
 'hieracium-morulum']

In [8]:
species_151 = som_151.species_slug.unique()

for species in species_151:
    try:
        som_151.loc[som_151.species_slug == species, 'BL'] = watch_lists.loc[species]['BL']
    except:
        som_151.loc[som_151.species_slug == species, 'BL'] = 0
    try:
        som_151.loc[som_151.species_slug == species, 'WL'] = watch_lists.loc[species]['WL']
    except:
        som_151.loc[som_151.species_slug == species, 'WL'] = 0
    try:
        som_151.loc[som_151.species_slug == species, 'RL'] = watch_lists.loc[species]['red_list']
    except:
        som_151.loc[som_151.species_slug == species, 'RL'] = 0
    try:
        som_151.loc[som_151.species_slug == species, "gbif"] = watch_lists.loc[species]["gbif"]
    except:
        som_151.loc[som_151.species_slug == species, 'gbif'] = 0
        
listed_observations = som_151.groupby('place').agg({"BL":"sum", "WL":"sum", "RL":"sum", "gbif":"sum"})


listed_observations.sum()

BL        62.0
WL         1.0
RL      1059.0
gbif    1160.0
dtype: float64